In [1]:
# main 
import numpy as np
import pandas as pd

# deal with warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# set path
path = '/kaggle/input/playground-series-s4e5'

# load data
train = pd.read_csv(f'{path}/train.csv', index_col=0)
test = pd.read_csv(f'{path}/test.csv', index_col=0)

In [3]:
train_features = train.drop(columns=['FloodProbability']).copy()

# PipeLine Maker

In [4]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                     StandardScaler())

preprocessing_baseline = make_column_transformer(
    (default_num_pipeline, make_column_selector(dtype_include=np.number)),
)

In [5]:
from catboost import CatBoostRegressor
# cat boost
model = make_pipeline(preprocessing_baseline,
                      CatBoostRegressor(verbose=0))

# New features

In [6]:
def stat_features(data, only_stat=False):
    
    df = data.copy()
    cols = df.columns
    
#     df['sum'] = df[cols].sum(axis=1) # Same info as mean 1 corr
    df['mean'] = df[cols].mean(axis=1)
    df['std'] = df[cols].std(axis=1)
    df['median'] = df[cols].median(axis=1)
    df['min_all'] = df[cols].min(axis=1)
    df['max_all'] = df[cols].max(axis=1)
    df['ptp'] = df[cols].values.ptp(axis=1)
    df['q25'] = df[cols].quantile(0.25, axis=1)
    df['q75'] = df[cols].quantile(0.75, axis=1)
    
    if only_stat:
        df = df.drop(columns=cols) # drops lables too
        
    return df

In [7]:
X_train = stat_features(train_features, only_stat=True)
X_test = stat_features(test, only_stat=True)

y_train = train[['FloodProbability']]

# PipeLine fit

In [8]:
model.fit(X_train, y_train)
test_pred = model.predict(X_test)

In [9]:
sub = pd.DataFrame({y_train.columns[0]: test_pred},
             index=test.index)
sub

,FloodProbability
id,
1117957,0.578547
1117958,0.457183
1117959,0.449342
1117960,0.467124
1117961,0.467543
...,...
1863257,0.475929
1863258,0.446555
1863259,0.621064


In [10]:
MAKE_SUBMISSION = True

if MAKE_SUBMISSION:
    sub.to_csv('submission.csv', index=True)